In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Colab/20212/Optimize_Project


/content/drive/MyDrive/Colab/20212/Optimize_Project


In [ ]:
DATA_FILE = './Data Input/data_v2/2005PisingerSigurd58.bpp'
with open(DATA_FILE,'r') as f:
  lines = f.readlines()
  N,K = [int(i) for i in lines[1].strip().split()[:2]]
  W_, H_ = [],[]
  h,w = [],[]
  c = []
  bins = lines[3].strip().split()
  costs = lines[4].strip().split()
  for i in range(K):
    H_.append(int(bins[2*i]))
    W_.append(int(bins[2*i+1]))
    c.append(int(costs[i]))
  for i in range(N):
    item = lines[i+5].strip().split()
    h.append(int(item[0]))
    w.append(int(item[1]))
  W, H = max(W_), max(H_)

In [ ]:
data = {'bin': {'h': H_, 'w': W_, 'c': c}}
N_ITEMS = 2
for i in range(N):
  data[f'cat{i}'] = {'w': w[i],'h':h[i],'items':N_ITEMS}

In [ ]:
data.keys()

dict_keys(['bin', 'cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18', 'cat19'])

In [ ]:
N,K,W_,H_,w,h,c,W,H

(20,
 5,
 [30, 20, 25, 22, 19],
 [30, 16, 27, 27, 27],
 [9, 4, 2, 9, 2, 6, 4, 6, 5, 5, 6, 10, 2, 1, 10, 4, 2, 6, 2, 8],
 [5, 1, 8, 4, 1, 4, 4, 1, 1, 4, 2, 9, 5, 3, 7, 7, 5, 6, 9, 7],
 [900, 204, 630, 532, 435],
 30,
 30)

In [ ]:
import time
class TimeCounter():
  def __init__(self):
    self.start_time = 0
    self.end_time = 0
  def start(self):
    self.start_time = time.time()
  def end(self):
    self.end_time = time.time()
  def get_duration(self):
    return self.end_time - self.start_time

In [ ]:
COUNTER = TimeCounter()

In [ ]:
import matplotlib.pyplot as plt

# Heuristic model

In [ ]:
!git clone https://github.com/solomon-b/greedypacker

fatal: destination path 'greedypacker' already exists and is not an empty directory.


In [ ]:
!pip install git+https://github.com/solomon-b/greedypacker.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/solomon-b/greedypacker.git to /tmp/pip-req-build-ujhq47n7
  Running command git clone -q https://github.com/solomon-b/greedypacker.git /tmp/pip-req-build-ujhq47n7


In [ ]:
from copy import deepcopy
import random
import greedypacker
import os
PENALTY_TERM = 1000
RUN_TIME = 5
NUM_ITEMS = 1

print("*"*50)
DATA_FILE = './Data Input/data_v2/2005PisingerSigurd58.bpp'
print("Solving", DATA_FILE)
print("*"*50)
with open(DATA_FILE,'r') as f:
  lines = f.readlines()
  N,K = [int(i) for i in lines[1].strip().split()[:2]]
  W_, H_ = [],[]
  h,w = [],[]
  c = []
  bins = lines[3].strip().split()
  costs = lines[4].strip().split()
  for i in range(K):
    H_.append(int(bins[2*i]))
    W_.append(int(bins[2*i+1]))
    c.append(int(costs[i]))
  for i in range(N):
    item = lines[i+5].strip().split()
    h.append(int(item[0]))
    w.append(int(item[1]))
  W, H = max(W_), max(H_)
data = {'bin': {'h': H_, 'w': W_, 'c': c}}
for i in range(N):
  data[f'cat{i}'] = {'w': w[i],'h':h[i],'items':NUM_ITEMS}
items_info = [(data[cat]['h'], data[cat]['w']) for cat in data if cat!='bin' for i in range(data[cat]['items'])]
bins_info = [(data['bin']['h'][i], data['bin']['w'][i], data['bin']['c'][i]) for i in range(len(data['bin']['h']))]
N = len(items_info)
K = len(bins_info)

## Tabu params
ITERS = 500
MAX_TABU_SIZE = 10
WAIT_SIZE = 10


class Solution:
  def __init__(self, solution=None):
    self.solution = solution
    self.bin_dict = {}
    self.fitness = None
    self.minimal_items = 1000
  def calculate_fitness(self):
    if self.fitness!=None:
      return self.fitness
    assert len(self.solution) == N
    fitness = 0
    for i in range(len(self.solution)):
      if self.solution[i] in self.bin_dict:
        self.bin_dict[self.solution[i]].append(i)
      else:
        self.bin_dict[self.solution[i]] = [i]
    for key, items in self.bin_dict.items():
      M = greedypacker.BinManager(bins_info[key][0], bins_info[key][1], pack_algo='guillotine', heuristic='best_longside', rectangle_merge=True, rotation=True)
      fitness+= bins_info[key][2]
      total_items = len(items)
      for item in items:
        Item = greedypacker.Item(items_info[item][0], items_info[item][1])
        M.add_items(Item)
      M.execute() 
      for bin in M.bins[1:]:
        for item in bin.items:
          fitness+= PENALTY_TERM*item.width*item.height
      if len(M.bins)<2:
        self.minimal_items = min(self.minimal_items, total_items)
    self.fitness = fitness
    return self.fitness
  def __str__(self):
    return "Encoding: {}, Fitness: {}".format(self.solution, self.calculate_fitness())
  def equal(self, solution):
    for i in range(N):
      if self.solution[i]!=solution.solution[i]:
        return False
    return True
  def initialize_solution(self):
    flag = [-1 for i in range(K)]
    free_space = [bins_info[j][0]*bins_info[j][1] for j in range(K)]
    solution = [-1 for i in range(N)]
    for i in range(N):
      candidate = []
      for j in range(K):
        if  items_info[i][0]<=bins_info[j][0] and items_info[i][1]<=bins_info[j][1]:
          #if free_space[j]-items_info[i][0]*items_info[i][1]<min_space and free_space[j]-items_info[i][0]*items_info[i][1]>0:
          #  min_space = free_space[j]-items_info[i][0]*items_info[i][1]
          #  solution[i] = j
          #  flag[j] = i
          if free_space[j]-items_info[i][0]*items_info[i][1]>0:
            candidate.append(j)
      try:
        solution[i] = random.choice(candidate)
      except:
        solution[i] = random.randint(0,K-1)
      free_space[solution[i]]-= items_info[i][0]*items_info[i][1]
    self.solution = deepcopy(solution)
    #self.solution = [3 for i in range(N)]
  def get_neighbour(self,flag):
    if flag:
      for i in range(N):
        tmp = deepcopy(self.solution)
        current = tmp[i]
        for k in range(K):
          if k!=current and items_info[i][0]<=bins_info[k][0] and items_info[i][1]<=bins_info[k][1]:
            tmp[i] = k
            yield Solution(deepcopy(tmp))
    else:
      for key, items in self.bin_dict.items():
        for k in range(K):
          tmp = deepcopy(self.solution)
          flag = 1
          for i in items:
              if k!=key and items_info[i][0]<=bins_info[k][0] and items_info[i][1]<=bins_info[k][1]:
                tmp[i] = k
              else:
                flag = 0
                break
          if flag!=0:
            yield Solution(deepcopy(tmp))
class Tabu:
  def __init__(self):
    self.tabu_list = []
  def push(self, solution : Solution):
    self.tabu_list.append(solution)
  def pop(self):
    self.tabu_list.pop(0)
  def contain(self, solution: Solution):
    for i in range(len(self.tabu_list)):
      if solution.equal(self.tabu_list[i]):
        return True
    return False
  def length(self):
    return len(self.tabu_list) 

COUNTER.start()

for i in range(RUN_TIME):
  print(f'RUNTIME {i}:\n------------')
  s_best = Solution()
  s_best.initialize_solution()
  best_candidate = s_best
  tabu_list = Tabu()
  tabu_list.push(best_candidate)
  print(s_best)
  wait = WAIT_SIZE-1
  ### Main Loop
  for i in range(ITERS):
    if wait>0:
      s_neighbours = list(best_candidate.get_neighbour(True))
    else:
      wait = WAIT_SIZE
      s_neighbours = list(best_candidate.get_neighbour(False))
    #s_neighbours = list(best_candidate.get_neighbour(True))
    best_candidate = s_neighbours[0]
    for s_candidate in s_neighbours:
      if not tabu_list.contain(s_candidate) and (s_candidate.calculate_fitness() < best_candidate.calculate_fitness() or (s_candidate.calculate_fitness() == best_candidate.calculate_fitness() and s_candidate.minimal_items<best_candidate.minimal_items)):
        best_candidate = s_candidate
        #if wait<WAIT_SIZE or (s_best.calculate_fitness()!=s_candidate.calculate_fitness()):
        #  best_candidate = s_candidate    
    if best_candidate.calculate_fitness() < s_best.calculate_fitness():
      s_best = best_candidate
    else:
      wait-= 1
    tabu_list.push(best_candidate)
    if tabu_list.length() > MAX_TABU_SIZE:
      tabu_list.pop()
  print(s_best)
  COUNTER.end()
  print("Solved in {:.2f}s".format(COUNTER.get_duration()))

**************************************************
Solving ./Data Input/data_v2/2005PisingerSigurd58.bpp
**************************************************
RUNTIME 0:
------------
Encoding: [0, 0, 4, 0, 4, 4, 2, 0, 0, 3, 1, 4, 4, 2, 1, 2, 4, 0, 1, 3], Fitness: 2701
Encoding: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], Fitness: 532
Solved in 51.33s
RUNTIME 1:
------------
Encoding: [3, 3, 0, 4, 0, 2, 1, 2, 2, 2, 1, 4, 4, 1, 4, 1, 1, 2, 4, 3], Fitness: 2701
Encoding: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], Fitness: 532
Solved in 102.38s
RUNTIME 2:
------------
Encoding: [4, 0, 2, 2, 1, 2, 3, 0, 0, 3, 3, 0, 4, 4, 2, 1, 0, 1, 2, 0], Fitness: 2701
Encoding: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], Fitness: 532
Solved in 152.93s
RUNTIME 3:
------------
Encoding: [1, 4, 4, 0, 0, 4, 0, 2, 4, 0, 1, 2, 1, 1, 2, 3, 0, 4, 2, 0], Fitness: 2701
Encoding: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], Fitness: 532
Solved in 2

In [ ]:
s_neighbours = list(s_best.get_neighbour(False))
for s_candidate in s_neighbours:
  print(s_candidate)

Encoding: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], Fitness: 900
Encoding: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Fitness: 195204
Encoding: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], Fitness: 630
Encoding: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], Fitness: 25435


In [ ]:
s_best.solution

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [ ]:
bins_info

[(30, 30, 900), (16, 20, 204), (27, 25, 630), (27, 22, 532), (27, 19, 435)]

# Genetic Algorithm

In [ ]:
import numpy as np
import random
import copy

M = greedypacker.BinManager(8, 4, pack_algo='guillotine', heuristic='best_longside', rectangle_merge=True, rotation=True)
PENALTY_TERM = 1000
INFINITY = 1e8
class Individual:
  def __init__(self, chromosome=None, init=False):
    self.chromosome = chromosome
    if init:
      self.chromosome = self.initialize_solution()
    self.bin_dict = {}
    self.fitness = self.calculate_fitness()

  def calculate_fitness(self):
    fitness = 0
    for i in range(N):
      if self.chromosome[i] in self.bin_dict:
        self.bin_dict[self.chromosome[i]].append(i)
      else:
        self.bin_dict[self.chromosome[i]] = [i]
    for key, items in self.bin_dict.items():
      M = greedypacker.BinManager(bins_info[key][0], bins_info[key][1], pack_algo='guillotine', heuristic='best_longside', rectangle_merge=True, rotation=True)
      fitness+= bins_info[key][2]
      oversize_items = 0
      for item in items:
        Item = greedypacker.Item(items_info[item][0], items_info[item][1])
        if items_info[item][0]>bins_info[key][0] or items_info[item][1]>bins_info[key][1]:
          fitness+= PENALTY_TERM*items_info[item][0]*items_info[item][1]
          continue
        M.add_items(Item)
      M.execute()
      for bin in M.bins[1:]:
        for item in bin.items:
          fitness+= PENALTY_TERM*item.width*item.height
    fitness = 1000.0/fitness
    return fitness

  def initialize_solution(self):
    flag = [-1 for i in range(K)]
    free_space = [bins_info[j][0]*bins_info[j][1] for j in range(K)]
    solution = [-1 for i in range(N)]
    for i in range(N):
      candidate_list = []
      for j in range(K):
        if  items_info[i][0]<=bins_info[j][0] and items_info[i][1]<=bins_info[j][1]:
          candidate_list.append(j)
      solution[i] = random.choice(candidate_list)
    return solution

  def __str__(self):
    return f'Chromosome: {self.chromosome} \t Fitness: {round(1000 / (self.fitness + 1e-30), ndigits=2)}'
  
  def __len__(self):
    return len(self.chromosome)

In [ ]:
class Population:
  def __init__(self, pop_size, chromosome_size, first_population=False):
    self.pop_size = pop_size
    self.chromosome_size = chromosome_size
    if first_population:
      self.list_individual = [Individual(init=True) for _ in range(pop_size)]
    else:
      self.list_individual = []

  def sort(self):
    return sorted(self.list_individual, key=lambda i: i.fitness, reverse=True)
  
  def normalize_fitness(self):
    list_fitness = [individual.fitness for individual in self.list_individual]
    sum_pop_fitness = sum(list_fitness)
    for fitness in list_fitness:
      fitness /= (sum_pop_fitness + 1e-30)
    
    return list_fitness

  def add_individual(self, individual: Individual):
    self.list_individual.append(individual)
  
  def evaluation(self, print_result=True):
    list_fitness = [individual.fitness for individual in self.list_individual]
    best_individual = self.list_individual[list_fitness.index(max(list_fitness))]
    worst_individual = self.list_individual[list_fitness.index(min(list_fitness))]

    list_real_fitness = [1000 / (fitness + 1e-30) for fitness in list_fitness]
    counter = Counter(list_real_fitness)
    if print_result:
      print('Best individual \t' + str(best_individual))
      print('Worst individual \t' + str(worst_individual))
      print(counter)
    else:
      return best_individual, worst_individual, counter
  
  def __len__(self):
    return len(self.list_individual)

In [ ]:
class GeneticAlgorithm:
  def __init__(self, chromosome_size, pop_size=100, GA_iter=1000, crossover_prob=0.8, mutation_prob=0.1, elitist_selection=0.1, use_local_search=False, hill_climbing_iter=50):
    self.chromosome_size = chromosome_size
    self.pop_size = pop_size
    self.max_iter = GA_iter
    self.crossover_prob = crossover_prob
    self.mutation_prob = mutation_prob
    self.use_local_search = use_local_search
    self.hill_climbing_iter = hill_climbing_iter
    self.elitist_selection = elitist_selection
  
  def roulette_wheel_selection(self, population):
    pop_fitness = population.normalize_fitness()
    cumsum = np.cumsum(pop_fitness)
    r = np.random.rand()
    if r > max(cumsum):
      index = random.randint(0, self.pop_size - 1)
    else:
      index = np.argwhere(r <= cumsum)[0][0]
    
    return population.list_individual[index]
  
  def pmx_crossover(self, parent_1, parent_2):
    [start, end] = sorted(random.sample(range(N), 2))
    clone_2 = copy.copy(parent_2.chromosome)
    clone_2[start:end+1] = parent_1.chromosome[start:end+1]
    clone_1 = copy.copy(parent_1.chromosome)
    clone_1[start:end+1] = parent_2.chromosome[start:end+1]
    
    child_1 = Individual(clone_1)
    child_2 = Individual(clone_2)

    return child_1, child_2

  def point_mutation(self, individual):
    chromosome = individual.chromosome
    N = self.chromosome_size
    #idx_1, idx_2 = random.sample(range(N), 2)
    #new_chromosome = copy.deepcopy(chromosome)
    #new_chromosome[idx_1] = chromosome[idx_2]
    #new_chromosome[idx_2] = chromosome[idx_1]
    i = random.randint(0,N-1)
    new_chromosome = copy.deepcopy(chromosome)
    candidate = []
    for j in range(K):
      if  items_info[i][0]<=bins_info[j][0] and items_info[i][1]<=bins_info[j][1]:
        candidate.append(j)
    new_chromosome[i] = random.choice(candidate)
    return Individual(new_chromosome)
  
  def bin_mutation(self, individual):
    chromosome = individual.chromosome
    N = self.chromosome_size
    #idx_1, idx_2 = random.sample(range(N), 2)
    #new_chromosome = copy.deepcopy(chromosome)
    #new_chromosome[idx_1] = chromosome[idx_2]
    #new_chromosome[idx_2] = chromosome[idx_1]
    new_chromosome = copy.deepcopy(chromosome)
    candidate = []
    while(len(candidate)==0):
      k = random.choice(list(individual.bin_dict.keys()))
      candidate = []
      for j in range(K):
        flag = 1
        for i in individual.bin_dict[k]:
          if items_info[i][0]<=bins_info[j][0] and items_info[i][1]<=bins_info[j][1] and j!=k:
            continue
          else:
            flag = 0
            break
        if flag!=0:
          candidate.append(j)
    replace = random.choice(candidate)
    for i in individual.bin_dict[k]:
      new_chromosome[i] = replace
    return Individual(new_chromosome)
  
  def compute(self, print_result=False):
    population = Population(pop_size=self.pop_size, chromosome_size=self.chromosome_size, first_population=True)
    for i in tqdm(range(self.max_iter)):
      new_population = Population(pop_size=self.pop_size, chromosome_size=self.chromosome_size, first_population=False)
      sorted_population = population.sort()
      new_population.list_individual.extend(sorted_population[:int(self.pop_size*self.elitist_selection)])
      # import pdb; pdb.set_trace()
      for j in range((self.pop_size - int(self.pop_size*self.elitist_selection)) // 2):
        parent_1 = self.roulette_wheel_selection(population)
        parent_2 = self.roulette_wheel_selection(population)
        prob = random.random()
        if prob < self.crossover_prob:
          individual_1, individual_2 = self.pmx_crossover(parent_1, parent_2)
          if prob <  self.mutation_prob:
            if random.random()<0.5:
              individual_1 = self.point_mutation(individual_1)
              individual_2 = self.point_mutation(individual_2)
            else:
              individual_1 = self.bin_mutation(individual_1)
              individual_2 = self.bin_mutation(individual_2)
          new_population.add_individual(individual_1)
          new_population.add_individual(individual_2)
        else:
          new_population.add_individual(parent_1)
          new_population.add_individual(parent_2)

      population = copy.deepcopy(new_population)
    if print_result:
      population.evaluation(print_result=print_result)
    else:
      return population.evaluation(print_result=print_result)[0]

In [ ]:
from tqdm.notebook import tqdm
import time
from collections import Counter
from statistics import stdev
class Evaluation:
  def __init__(self, genetic_algorithm, n_running=100, print_result=True, save_result=True):
    self.genetic_algorithm = genetic_algorithm
    self.n_running = n_running
    self.print_result_bool = print_result
    self.save_result_bool = save_result
    self.list_best_result = []
    self.list_running_time = []

  def running(self):
    for i in tqdm(range(self.n_running)):
      start_time = time.time()
      self.list_best_result.append(self.genetic_algorithm.compute())
      self.list_running_time.append(time.time() - start_time)
    
    if self.save_result_bool:
      self.save_result(use_local_search=self.genetic_algorithm.use_local_search)
    if self.print_result_bool:
      self.print_result()

  def save_result(self, use_local_search=False, path='/content/drive/MyDrive/Colab/20212/Optimize_Project/GA_Result/'):
    ga = self.genetic_algorithm
    N = ga.chromosome_size
    list_fitness = [round(100 / (individual.fitness + 1e-30), ndigits=2) for individual in self.list_best_result]
    if use_local_search:
      output_file = 'GA+LocalSearch_N={}_popSize={}_iter={}_crossoverProb={}_mutationProb={}_hillClimbingIter={}.txt'.format(ga.chromosome_size, ga.pop_size, ga.max_iter, ga.crossover_prob, ga.mutation_prob, ga.hill_climbing_iter)
    else:
      output_file = 'GA_N={}_popSize={}_iter={}_crossoverProb={}_mutationProb={}.txt'.format(ga.chromosome_size, ga.pop_size, ga.max_iter, ga.crossover_prob, ga.mutation_prob)
    
    with open(file=path+output_file, mode='w', encoding='utf-8') as f:
      f.write(f'Average running time: {round(sum(self.list_running_time) / len(self.list_running_time), ndigits=4)}\n')
      f.write('Best result: \t' + str(self.list_best_result[list_fitness.index(min(list_fitness))]) + '\t Freq: ' + str(Counter(list_fitness)[min(list_fitness)]) +'\n')
      f.write('Worst result: \t' + str(self.list_best_result[list_fitness.index(max(list_fitness))]) + '\t Freq: ' + str(Counter(list_fitness)[max(list_fitness)]) + '\n')
      f.write('Mode result: \t' + str(self.list_best_result[list_fitness.index(max(Counter(list_fitness).items(), key=lambda x: x[1])[0])]) + '\t Freq: ' + str(Counter(list_fitness)[max(Counter(list_fitness).items(), key=lambda x: x[1])[0]]) + '\n')
      f.write('Average fitness: {}'.format(round(sum(list_fitness) / len(list_fitness), ndigits=4)) + '\n')
      f.write('Standard deviation: {}'.format(round(stdev(list_fitness), ndigits=4)) + '\n')
      f.write(str(Counter(list_fitness)))

  def print_result(self):
    print(f'Average running time: {round(sum(self.list_running_time) / len(self.list_running_time), ndigits=4)}')
    list_fitness = [round(100 / (individual.fitness + 1e-30), ndigits=2) for individual in self.list_best_result]
    print('Best result: \t' + str(self.list_best_result[list_fitness.index(min(list_fitness))]) + '\t Freq: ' + str(Counter(list_fitness)[min(list_fitness)]))
    print('Worst result: \t' + str(self.list_best_result[list_fitness.index(max(list_fitness))]) + '\t Freq: ' + str(Counter(list_fitness)[max(list_fitness)]))
    print('Mode result: \t' + str(self.list_best_result[list_fitness.index(max(Counter(list_fitness).items(), key=lambda x: x[1])[0])]) + '\t Freq: ' + str(Counter(list_fitness)[max(Counter(list_fitness).items(), key=lambda x: x[1])[0]]))
    print('Average fitness: {}'.format(round(sum(list_fitness) / len(list_fitness), ndigits=4)))
    print('Standard deviation: {}'.format(round(stdev(list_fitness), ndigits=4)))
    print(Counter(list_fitness))

In [ ]:
population = Population(pop_size=100, chromosome_size=N, first_population=True)

In [ ]:
M = greedypacker.BinManager(8, 4, pack_algo='guillotine', heuristic='best_longside', rectangle_merge=True, rotation=True)
PENALTY_TERM = 1000
INFINITY = 1e8

genetic_algorithm = GeneticAlgorithm(chromosome_size=N, use_local_search=False, GA_iter=1000)
evaluation = Evaluation(genetic_algorithm=genetic_algorithm, n_running=2, save_result=True)
evaluation.running()

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Average running time: 130.8244
Best result: 	Chromosome: [1, 4, 1, 4, 4, 4, 4, 4, 1, 4, 4, 4, 1, 4, 1, 1, 4, 1, 4, 4] 	 Fitness: 639.0	 Freq: 2
Worst result: 	Chromosome: [1, 4, 1, 4, 4, 4, 4, 4, 1, 4, 4, 4, 1, 4, 1, 1, 4, 1, 4, 4] 	 Fitness: 639.0	 Freq: 2
Mode result: 	Chromosome: [1, 4, 1, 4, 4, 4, 4, 4, 1, 4, 4, 4, 1, 4, 1, 1, 4, 1, 4, 4] 	 Fitness: 639.0	 Freq: 2
Average fitness: 63.9
Standard deviation: 0.0
Counter({63.9: 2})


In [ ]:
1000/evaluation.list_best_result[0].fitness

245.99999999999997

# CP model 1

In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.5 MB 18.3 MB/s 
     |████████████████████████████████| 407 kB 50.8 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220527125636 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.2 which is incompatible.
tensorflow-metadata 1.8.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.2 which is incompatible.
googleapis-common-protos 1.56.2 requires protobuf<4.0.0dev,>=3.15.0, but you have protobuf 4.21.2 which is incompatible.
google-cloud-bigquery-storage 1.1.2 requires protobuf<4.0.0dev, but you have protobuf 4.21.2 wh

In [ ]:
from ortools.sat.python import cp_model

#---------------------------------------------------
# data 
#---------------------------------------------------

#data = {'bin':{'h':[30,30,60,50,40],'w':[20,10,30,40,20], 'c': [10,10,30,20,1]}}     
#for i in range(N):
#  data[f'cat{i}'] = {'w': w[i],'h':h[i],'items':1}
#
# extract data for easier access
#
def run_CP():
  # bin width and height
  H = data['bin']['h']
  C = data['bin']['c']
  #W = data['bin']['w']
  W = [0]
  for i in range(len(data['bin']['h'])):
    W.append(W[i]+data['bin']['w'][i])
  # h,w,cat for each item
  h = [data[cat]['h'] for cat in data if cat!='bin' for i in range(data[cat]['items'])]
  w = [data[cat]['w'] for cat in data if cat!='bin' for i in range(data[cat]['items'])]
  cat = [cat for cat in data if cat!='bin' for i in range(data[cat]['items'])]
  n = len(h)  # number of items
  m = len(data['bin']['h'])      # number of bins

  #---------------------------------------------------
  # or-tools model 
  #---------------------------------------------------


  model = cp_model.CpModel()

  #
  # variables
  #

  # x1,x2 and y1,y2 are start and end
  x1 = [model.NewIntVar(0, W[-1]-w[i], f'x1[{i}]') for i in range(n)]
  x2 = [model.NewIntVar(w[i], W[-1], f'x2[{i}]') for i in range(n)]
  y1 = [model.NewIntVar(0, max(H) - h[i], f'y1[{i}]') for i in range(n)]
  y2 = [model.NewIntVar(h[i], max(H), f'y2[{i}]') for i in range(n)]

  # interval variables
  lit = [[model.NewBoolVar(f'lit[{i}][{j}]') for j in range(m)] for i in range(n)]
  # Box using
  u = [model.NewBoolVar(f'u[{i}]') for i in range(m)]


  #
  # constraints
  #
  #for i in range(n):
  #  #model.Add(x1[i] == x[i] + b[i]*W)
  #  model.Add(x2[i] == x1[i] + w[i])
  #  model.Add(y2[i] == y1[i] + h[i])
  for i in range(n):
    model.Add(sum(lit[i]) == 1)
    model.Add(sum(W[j] * lit[i][j] for j in range(m)) <= x1[i])
    model.Add(sum(W[j + 1] * lit[i][j] for j in range(m)) >= x2[i])
    model.Add(x1[i]+w[i] == x2[i])
    #model.Add(sum(H[j] * lit[i][j] for j in range(m)) <= x1[i])
    model.Add(y1[i]+h[i] == y2[i])
    model.Add(sum(H[j] * lit[i][j] for j in range(m)) >= y2[i])
  # no overlap for items in same bin
  x_interval = [model.NewIntervalVar(x1[i], w[i], x2[i], f'x_interval[{i}]') for i in range(n)]
  y_interval = [model.NewIntervalVar(y1[i], h[i], y2[i], f'y_interval[{i}]') for i in range(n)]
  model.AddNoOverlap2D(x_interval, y_interval)
  #obj = model.NewIntVar(0, sum(C), 'obj')
  #model.Add(obj>=sum(u[j]*C[j] for j in range(m)))
  for j in range(m):
      model.Add(sum(lit[i][j] for i in range(n))>0).OnlyEnforceIf(u[j])
      model.Add(sum(lit[i][j] for i in range(n))<=0).OnlyEnforceIf(u[j].Not())
  obj = model.NewIntVar(0,sum(C),'obj')
  model.Add(obj==sum(u[j]*C[j] for j in range(m)))
  model.Minimize(obj)
  # M = solver.IntVar(0.0, infinity, 'M')
  solver = cp_model.CpSolver()
  solver.parameters.num_search_workers = 8
  solver.parameters.max_time_in_seconds = 300.0
  COUNTER.start()
  status = solver.Solve(model)
  COUNTER.end()
  res = []
  print("Solved in {:.2f} s".format(COUNTER.get_duration()))
  print("Objective value ",solver.Value(obj))
  for i in range(n):
    res.append([])
    for j in range(m):
      res[i].append(solver.Value(lit[i][j]))
  print(solver.StatusName())
  print(res)

In [ ]:
run_CP()

Solved in 2.30 s
Objective value  571
OPTIMAL
[[0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0]]


# Experiments

In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.5 MB 4.7 MB/s 
     |████████████████████████████████| 407 kB 56.1 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220527125636 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.2 which is incompatible.
tensorflow-metadata 1.8.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.2 which is incompatible.
googleapis-common-protos 1.56.2 requires protobuf<4.0.0dev,>=3.15.0, but you have protobuf 4.21.2 which is incompatible.
google-cloud-bigquery-storage 1.1.2 requires protobuf<4.0.0dev, but you have protobuf 4.21.2 whi

In [ ]:
%cd ./Data Input/

/content/drive/MyDrive/Colab/20212/Optimize_Project/Data Input


In [ ]:
%%writefile main.py
import os
import logging
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model
import time

class TimeCounter():
  def __init__(self):
    self.start_time = 0
    self.end_time = 0
  def start(self):
    self.start_time = time.time()
  def end(self):
    self.end_time = time.time()
  def get_duration(self):
    return self.end_time - self.start_time

class Experiment_Manager():
  def read_data(self, file_name):
    #DATA_FILE = './Data Input/2005PisingerSigurd1.bpp'
    rootLogger.info("Read data from: {:s}".format(file_name))
    with open(file_name,'r') as f:
      lines = f.readlines()
      N,K = [int(i) for i in lines[1].strip().split()[:2]]
      W_, H_ = [],[]
      h,w = [],[]
      c = []
      bins = lines[3].strip().split()
      costs = lines[4].strip().split()
      for i in range(K):
        H_.append(int(bins[2*i]))
        W_.append(int(bins[2*i+1]))
        c.append(int(costs[i]))
      for i in range(N):
        item = lines[i+5].strip().split()
        h.append(int(item[0]))
        w.append(int(item[1]))
    W, H = max(W_), max(H_)
    data = {'bin': {'h': H_, 'w': W_, 'c': c}}
    for i in range(N):
      data[f'cat{i}'] = {'w': w[i],'h':h[i],'items':N_ITEMS}
    return data
  def run_MIPS(self, data):
    rootLogger.info("-----------Mixed Integer Programming------------")
    ### MIPS parameters initialize
    h = [data[cat]['h'] for cat in data if cat!='bin' for i in range(data[cat]['items'])]
    w = [data[cat]['w'] for cat in data if cat!='bin' for i in range(data[cat]['items'])]
    W_ = data['bin']['w']
    H_ = data['bin']['h']
    N = len(w)
    K = len(W_)
    W = max(W_)
    H = max(H_)
    c = data['bin']['c']

    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')
    solver.SetTimeLimit(TIME_LIMIT*1000)

    # Initialize variables
    #Variable

    infinity = solver.infinity()

    # Bien xi, yi low-left.
    x = {}
    y = {}
    z = {}
    for i in range(N):
        x[i] = solver.IntVar(0, infinity,f'x[{i}]') 
    for i in range(N):
        y[i] = solver.IntVar(0, infinity,f'y[{i}]')
    #Decision variable for using bin/ trunk
    for k in range(K):
        z[k] = solver.IntVar(0, 1, f'z[{k}]')

    # biến l_ij, b_ij, f_ik, 
    l ={}
    for i in range(N):
        for j in range(N):
            if (i != j):
              l[i,j] = solver.IntVar(0, 1, f'l[{i},{j}]')
    b = {}
    for i in range(N):
        for j in range(N):
            if (i != j):
                b[i,j] = solver.IntVar(0, 1, f'b[{i},{j}]')

    f = {}
    for i in range(N):
        for k in range(K):
            f[i,k] = solver.IntVar(0, 1, f'f[{i},{k}]')

    rootLogger.info('Number of variables = {:d}'.format(solver.NumVariables()))

    ### Initialize constraints
    #Define constraint
    # Non overlab constraint
    for i in range(N):
        for j in range(i+1, N):
            for k in range(K):
                solver.Add(l[i,j]+l[j,i]+b[i,j]+b[j,i]+(1-f[i,k])+(1-f[j,k])>=1)

    # neu i nằm bên trái j và  thì l_ij = 1 thif xi + wi <= xj ( Và có sự ngầm hiểu là i, j cùng bin)
    # Nếu i, j ko cùng bin thì lij  = 0, và vẫn đúng vì W là max with của tất cả các bin
    for i in range(N):
        for j in range(N):
            if (i !=j):
                solver.Add(x[i]-x[j]+W*l[i,j]<=W-w[i])
    for i in range(N):
        for j in range(N):
            if (i !=j):
                solver.Add(y[i]-y[j]+H*b[i,j]<=H-h[i])

    # Ko item nào có kích cỡ vượt quá kích cỡ của bin cả?.
    for i in range(N):
        for k in range(K):
            solver.Add(x[i]<=W_[k]-w[i]+(1-f[i,k])*W)
            solver.Add(y[i]<=H_[k]-h[i]+(1-f[i,k])*H)     
    # Constraint mỗi item thì phải thuộc đúng 1 chiếc xe tải?.
    for i in range(N):
            solver.Add(solver.Sum(f[i,k] for k in range(K)) == 1) #???. >= 1 hay == 1?

    # Điều kiện cuối cùng lq đến biến của hàm mục tiêu la biến Z_k
    for i in range(N):
        for k in range(K):
            solver.Add(z[k]>=f[i,k])
    rootLogger.info('Number of constraints = {:d}'.format(solver.NumConstraints()))

    ### Define objective function
    solver.Minimize(sum(z[k]*c[k] for k in range(K)))

    ### Solve
    COUNTER.start()
    status = solver.Solve()
    COUNTER.end()

    ### Display and log results
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        rootLogger.info("Solved in {:.2f}s".format(COUNTER.get_duration()))
        rootLogger.info('Objective value = {:d}'.format(int(solver.Objective().Value())))
        if status == 0:
          rootLogger.info("OPTIMAL SOLUTION")
        else:
          rootLogger.info("FEASIBLE SOLUTION")
      # for k in range(K):
      #     print(z[k].solution_value())
        inBin = []
        for i in range(N):
            for k in range(K):
                if (f[i,k].solution_value() == 1):
                    inBin.append(k)
                    #print('item',i,k)
        rootLogger.info(inBin)

  def run_CP(self, data):
    rootLogger.info("-----------Constraint Programming------------")
    # bin width and height
    H = data['bin']['h']
    C = data['bin']['c']
    #W = data['bin']['w']
    W = [0]
    for i in range(len(data['bin']['h'])):
      W.append(W[i]+data['bin']['w'][i])
    # h,w,cat for each item
    h = [data[cat]['h'] for cat in data if cat!='bin' for i in range(data[cat]['items'])]
    w = [data[cat]['w'] for cat in data if cat!='bin' for i in range(data[cat]['items'])]
    cat = [cat for cat in data if cat!='bin' for i in range(data[cat]['items'])]
    n = len(h)  # number of items
    m = len(data['bin']['h'])      # number of bins

    #---------------------------------------------------
    # or-tools model 
    #---------------------------------------------------


    model = cp_model.CpModel()

    #
    # variables
    #

    # x1,x2 and y1,y2 are start and end
    x1 = [model.NewIntVar(0, W[-1]-w[i], f'x1[{i}]') for i in range(n)]
    x2 = [model.NewIntVar(w[i], W[-1], f'x2[{i}]') for i in range(n)]
    y1 = [model.NewIntVar(0, max(H) - h[i], f'y1[{i}]') for i in range(n)]
    y2 = [model.NewIntVar(h[i], max(H), f'y2[{i}]') for i in range(n)]

    # interval variables
    lit = [[model.NewBoolVar(f'lit[{i}][{j}]') for j in range(m)] for i in range(n)]
    # Box using
    u = [model.NewBoolVar(f'u[{i}]') for i in range(m)]


    #
    # constraints
    #
    #for i in range(n):
    #  #model.Add(x1[i] == x[i] + b[i]*W)
    #  model.Add(x2[i] == x1[i] + w[i])
    #  model.Add(y2[i] == y1[i] + h[i])
    for i in range(n):
      model.Add(sum(lit[i]) == 1)
      model.Add(sum(W[j] * lit[i][j] for j in range(m)) <= x1[i])
      model.Add(sum(W[j + 1] * lit[i][j] for j in range(m)) >= x2[i])
      model.Add(x1[i]+w[i] == x2[i])
      #model.Add(sum(H[j] * lit[i][j] for j in range(m)) <= x1[i])
      model.Add(y1[i]+h[i] == y2[i])
      model.Add(sum(H[j] * lit[i][j] for j in range(m)) >= y2[i])
    # no overlap for items in same bin
    x_interval = [model.NewIntervalVar(x1[i], w[i], x2[i], f'x_interval[{i}]') for i in range(n)]
    y_interval = [model.NewIntervalVar(y1[i], h[i], y2[i], f'y_interval[{i}]') for i in range(n)]
    model.AddNoOverlap2D(x_interval, y_interval)
    #obj = model.NewIntVar(0, sum(C), 'obj')
    #model.Add(obj>=sum(u[j]*C[j] for j in range(m)))
    for j in range(m):
        model.Add(sum(lit[i][j] for i in range(n))>0).OnlyEnforceIf(u[j])
        model.Add(sum(lit[i][j] for i in range(n))<=0).OnlyEnforceIf(u[j].Not())
    obj = model.NewIntVar(0,sum(C),'obj')
    model.Add(obj==sum(u[j]*C[j] for j in range(m)))
    model.Minimize(obj)
    # M = solver.IntVar(0.0, infinity, 'M')
    solver = cp_model.CpSolver()
    solver.parameters.num_search_workers = 8
    solver.parameters.max_time_in_seconds = TIME_LIMIT
    COUNTER.start()
    status = solver.Solve(model)
    COUNTER.end()
    res = []
    rootLogger.info("Solved in {:.2f}s".format(COUNTER.get_duration()))
    rootLogger.info("Objective value = {:d}".format(int(solver.Value(obj))))
    for i in range(n):
      for j in range(m):
        if solver.Value(lit[i][j]) == 1:
          res.append(j)
    rootLogger.info(solver.StatusName())
    rootLogger.info(res)
  
  def run(self, data_file, MIP = True, CP = True):
      rootLogger.info("###################################")
      data = self.read_data(data_file)
      if MIP:
        self.run_MIPS(data)
      if CP:
        self.run_CP(data)

if __name__ == '__main__':
  TIME_LIMIT = 300
  N_ITEMS = 1

  COUNTER = TimeCounter()
  manager = Experiment_Manager()

  logFormatter = logging.Formatter("%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s")
  rootLogger = logging.getLogger()
  rootLogger.setLevel(logging.INFO)

  fileHandler = logging.FileHandler("{0}/{1}.log".format('./', f'experiments_N_ITEMS_{N_ITEMS}'), mode = 'w')
  fileHandler.setFormatter(logFormatter)
  rootLogger.addHandler(fileHandler)

  consoleHandler = logging.StreamHandler()
  consoleHandler.setFormatter(logFormatter)
  rootLogger.addHandler(consoleHandler)

  rootLogger.info("Experiment start")
  for file_name in os.listdir('./data_v2'):
    #DATA_FILE = './Data Input/2005PisingerSigurd.bpp'
    try:
      manager.run(os.path.join('./data_v2/',file_name))
    except:
      continue

Overwriting main.py


In [ ]:
%cd /content/drive/MyDrive/Colab/20212/Optimize_Project
%cd ./Data Input/

/content/drive/MyDrive/Colab/20212/Optimize_Project
/content/drive/MyDrive/Colab/20212/Optimize_Project/Data Input


In [ ]:
!python main.py

2022-07-11 09:34:09,560 [MainThread  ] [INFO ]  Experiment start
2022-07-11 09:34:09,561 [MainThread  ] [INFO ]  ###################################
2022-07-11 09:34:09,561 [MainThread  ] [INFO ]  Read data from: ./data_v2/2005PisingerSigurd4.bpp
2022-07-11 09:34:09,894 [MainThread  ] [INFO ]  -----------Mixed Integer Programming------------
2022-07-11 09:34:09,904 [MainThread  ] [INFO ]  Number of variables = 3405
2022-07-11 09:34:10,162 [MainThread  ] [INFO ]  Number of constraints = 7660
2022-07-11 09:39:10,256 [MainThread  ] [INFO ]  -----------Constraint Programming------------
2022-07-11 09:44:10,317 [MainThread  ] [INFO ]  Solved in 300.03s
2022-07-11 09:44:10,317 [MainThread  ] [INFO ]  Objective value = 204
2022-07-11 09:44:10,318 [MainThread  ] [INFO ]  FEASIBLE
2022-07-11 09:44:10,318 [MainThread  ] [INFO ]  [1, 3, 3, 3, 3, 3, 4, 4, 0, 0, 3, 4, 0, 0, 3, 1, 1, 4, 3, 0, 3, 1, 0, 1, 0, 4, 4, 3, 1, 4, 1, 1, 1, 1, 1, 4, 0, 0, 0, 0]


# Example bin packing 2d

In [ ]:

from ortools.sat.python import cp_model
import pandas

#---------------------------------------------------
# data 
#---------------------------------------------------

data = {'bin':{'h':[60,30,20],'w':[40,20,20]},
        'cat1':{'w': 7,'h':12,'items':10},
        'cat2':{'w': 9,'h': 3,'items':5},
        'cat3':{'w': 5,'h':14,'items':5},
        'cat4':{'w':13,'h': 9,'items':10},
        'cat5':{'w': 6,'h': 8,'items': 5},
        'cat6':{'w':20,'h': 5,'items': 5}}      

#
# extract data for easier access
#

# bin width and height
H = data['bin']['h']
W = data['bin']['w']

# h,w,cat for each item
h = [data[cat]['h'] for cat in data if cat!='bin' for i in range(data[cat]['items'])]
w = [data[cat]['w'] for cat in data if cat!='bin' for i in range(data[cat]['items'])]
cat = [cat for cat in data if cat!='bin' for i in range(data[cat]['items'])]
n = len(h)  # number of items
m = 10  # max number of bins

#---------------------------------------------------
# or-tools model 
#---------------------------------------------------

model = cp_model.CpModel()

#
# variables
#

# x and y
x = [model.NewIntVar(0,W-w[i],f'x{i}') for i in range(n)]

xb1 = [model.NewIntVar(0,m*W-w[i],f'xb1.{i}') for i in range(n)]
xb2 = [model.NewIntVar(w[i],m*W,f'xb2.{i}') for i in range(n)]

y1 = [model.NewIntVar(0,H-h[i],f'y1.{i}') for i in range(n)]
y2 = [model.NewIntVar(h[i],H,f'y2.{i}') for i in range(n)]

# interval variables
xival = [model.NewIntervalVar(xb1[i],w[i],xb2[i],f'xival{i}') for i in range(n)]
yival = [model.NewIntervalVar(y1[i],h[i],y2[i],f'yival{i}') for i in range(n)]

# bin numbers
b = [model.NewIntVar(0,m-1,f'b{i}') for i in range(n)]

# objective
z = model.NewIntVar(0,m-1,'z')

#
# constraints
#
for i in range(n):
  model.Add(xb1[i] == x[i] + b[i]*W)
  model.Add(xb2[i] == xb1[i] + w[i])

model.AddNoOverlap2D(xival,yival)

model.AddMaxEquality(z,[b[i] for i in range(n)])

# objective
model.Minimize(z)    

#
# solve model
#
solver = cp_model.CpSolver()
# log does not work inside a Jupyter notebook
solver.parameters.log_search_progress = True
rc = solver.Solve(model)
print(f"return code:{rc}")
print(f"status:{solver.StatusName()}")

if rc == 4:
    df = pandas.DataFrame({ 
        'bin' : [solver.Value(b[i]) for i in range(n)],
        'x'   : [solver.Value(x[i]) for i in range(n)],
        'y'   : [solver.Value(y1[i]) for i in range(n)],
        'w'   : w,
        'h'   : h})
    display(df)

TypeError: ignored